In [6]:
!pip install evaluate

In [27]:
import pandas as pd
import numpy as np
import os
import random
import evaluate
from tqdm import tqdm

In [28]:
TRAIN_PATH = '/mnt/elice/dataset/train/'
TEST_PATH = '/mnt/elice/dataset/test/'

In [29]:
df = pd.read_csv(TRAIN_PATH + 'texts.csv')
raw_submission = pd.read_csv('raw_submission.csv')

In [30]:
import re

def clean_text(text, remove_space=True):
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\]','', text)
    if remove_space:
        text = ''.join(text.split())
    return text

In [33]:
targets = list(map(lambda x : clean_text(x), set(df['text'].tolist()))) #train label값
preds = raw_submission['text'].apply(lambda x : clean_text(x))# predict 값
threshold = 41 # text길이
cer_threshold = 0.5
metric = evaluate.load('cer')
for i in range(len(preds)):
    pred = preds[i]
    if len(pred) >= threshold:
        continue
    min_cer = 1e8
    index = -1
    for j in tqdm(range(len(targets))):
        target = targets[j]
        if len(target) >= threshold:
            continue
        cer = metric.compute(predictions=[pred], references=[target])
        if min_cer > cer:
            min_cer = cer
            index = j
    if min_cer < cer_threshold:
        preds[i] = targets[index]

  0%|          | 0/6000 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
submission = raw_submission
submission['text'] = preds

In [ ]:
submission.to_csv('submission.csv', index=False)